In [3]:
FN = 'train'

In [4]:
import os
# os.environ['THEANO_FLAGS'] = 'device=cpu,floatX=float32'

In [5]:
import keras
keras.__version__

Using TensorFlow backend.


'1.0.4'

Use indexing of tokens from vocabulary-embedding this does not clip the indexes of the words to vocab_size.

Use the index of outside words to replace them with several oov words (oov , oov0, oov1, ...) that appear in the same description and headline. This will allow headline generator to replace the oov with the same word in the description

In [6]:
FN0 = 'vocabulary-embedding'



implement the "simple" model from http://arxiv.org/pdf/1512.01712v1.pdf

you can start training from a pre-existing model. This allows you to run this notebooks many times, each time using different parameters and passing the end result of one run to be the input of the next.

I've started with maxlend=0 (see below) in which the description was ignored. I then moved to start with a high LR and the manually lowering it. I also started with nflips=0 in which the original headlines is used as-is and slowely moved to 12 in which half the input headline was fliped with the predictions made by the model (the paper used fixed 10%)

In [7]:
FN1 = 'train'

input data (X) is made from maxlend description words followed by eos followed by headline words followed by eos if description is shorter than maxlend it will be left padded with empty if entire data is longer than maxlen it will be clipped and if it is shorter it will be right padded with empty.

labels (Y) are the headline words followed by eos and clipped or padded to maxlenh

In other words the input is made from a maxlend half in which the description is padded from the left and a maxlenh half in which eos is followed by a headline followed by another eos if there is enough space.

The labels match only the second half and the first label matches the eos at the start of the second half (following the description in the first half)

In [8]:
maxlend=128 # 0 - if we dont want to use description at all
maxlenh=16
maxlen = maxlend + maxlenh
rnn_size = 64 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

the out of the first activation_rnn_size nodes from the top LSTM layer will be used for activation and the rest will be used to select predicted word

In [9]:
activation_rnn_size = 40 if maxlend else 0

In [10]:
# training parameters
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=16
nflips=10

In [11]:
nb_train_samples_per_epoch = 512
nb_val_samples_per_epoch = 32

In [12]:
import pickle

with open('data/%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
    
vocab_size, embedding_size = embedding.shape


In [13]:
with open('data/%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

In [14]:
nb_unknown_words = 10

In [15]:
print('number of examples',len(X),len(Y))
print('dimension of embedding space for words',embedding_size)
print('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words)
print('total number of different words',len(idx2word), len(word2idx))
print('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))



number of examples 25148 25148
dimension of embedding space for words 100
vocabulary size 40000 the last 10 words can be used as place holders for unknown/oov words
total number of different words 184531 184531
number of words outside vocabulary which we can substitue using glove similarity 27176
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) 117355


In [16]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i



when printing mark words outside vocabulary with ^ at their end

In [17]:
oov0 = vocab_size-nb_unknown_words

In [18]:
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [19]:
from sklearn.cross_validation import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples_per_epoch, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

/Users/MA573RWARR10R/anaconda/envs/pwsankaf/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(25116, 25116, 32, 32)

In [20]:
del X
del Y


In [21]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [22]:
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys


In [23]:
def prt(label, x):
    s = ""    
    for w in x:
        s += idx2word[w] + " "
    
    print(label + ': ' + s)

In [24]:
i = 334
prt('H',Y_train[i])
prt('D',X_train[i])

H: Meet Merrick Garland President Obama nominee for the Supreme Court 
D: April 1995 the first reports came about bombing Oklahoma City some the federal government top prosecutors gathered the office Merrick Garland who the time was the the Justice Department Somebody turned the saw bodies those young children being pulled out the rubble was enormously moving many levels including parents young children recalled Jamie Gorelick Garland boss All told children died the explosion the Alfred Murrah^ Federal Building most them killed the building center they watched Gorelick said Garland told her You need send there The request was granted Caption Before Merrick Garland was announced President Obama nominee the Supreme Court was the linchpin the 1995 Oklahoma City bombing investigation and ultimate convictions Timothy McVeigh and Terry Nichols Here look some Merrick Garland high profile work the Oklahoma City bombing investigation Top left Nicholas background Bill 


In [25]:
i = 8
prt('H',Y_test[i])
prt('D',X_test[i])

H: LISTEN Victim Doctors Describe^ War Zone Following Shootings^ Orlando 
D: victim and his doctors described war zone following the deadliest mass public shooting modern United States history Dr. Chadwick^ Smith surgeon the Orlando Regional Medical Center Orlando Fla. said that little after Sunday patients began arriving into the emergency room was quickly filled capacity with people suffering with wounds the extremities^ the chest the pelvis and the abdomen Some had small wounds others had wounds Angel Colon was Pulse Orlando nightclub when the shooting happened all fresh said Tuesday that was hesitant share his story But said wants people know his story they can understand how heartless one person could Colon said that soon heard the gunshots and his friends ran But was shot three times ran hit the floor was trampled could hear the shots could hear the pleas for help could just see him shooting everyone and can hear the shots getting closer and look over and shoots the girl next sai

In [26]:
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Dense, Activation, Dropout, RepeatVector, Merge
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2


In [27]:
# seed weight initialization
import numpy as np

random.seed(seed)
np.random.seed(seed)

In [28]:
regularizer = l2(weight_decay) if weight_decay else None
print(regularizer)

None


In [29]:
from termcolor import colored


def inspect_model(model):
    """Print the structure of Keras `model`."""
    layers_weights_shapes = []
    layers_names = []

    for i, l in enumerate(model.layers):
        layers_names.append('{} cls:{:30} name:{:30}'.format(i, colored(type(l).__name__, 'green'), colored(l.name, 'red')))

        _l_weights = l.get_weights()
        _l_shapes = []
        for i in _l_weights:
            _l_shapes.append(i.shape)
        layers_weights_shapes.append(_l_shapes)

    print("-" * 88)
    print("{:20}".format("Model Description By Layers"))
    print("-" * 88)
    for ln, lw in zip(layers_names, layers_weights_shapes):
        _w_string = ""
        for w in lw:
            _w_string += str(w) + " -- "

        print("{} {}".format(ln, _w_string[:-4]))
    print("-" * 88)

In [30]:
model = Sequential()

model.add(Embedding(vocab_size, 
                    embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer,
                    dropout=p_emb,
                    weights=[embedding], 
                    mask_zero=True,
                    name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

A special layer that reduces the input just to its headline part (second half). For each word in this part it concatenate the output of the previous layer (RNN) with a weighted average of the outputs of the description part. In this only the last rnn_size - activation_rnn_size are used from each output. The first activation_rnn_size output is used to computer the weights for the averaging.

In [31]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def get_output_shape_for(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [32]:
if activation_rnn_size:
    model.add(SimpleContext(name='simplecontext_1'))
    
model.add(TimeDistributed(Dense(vocab_size,
                                W_regularizer=regularizer, b_regularizer=regularizer,
                                name = 'timedistributed_1')))
model.add(Activation('softmax', name='activation_1'))

In [33]:
inspect_model(model)

----------------------------------------------------------------------------------------
Model Description By Layers
----------------------------------------------------------------------------------------
0 cls:Embedding             name:embedding_1           (40000, 100)
1 cls:LSTM                  name:lstm_1                (100, 64) -- (64, 64) -- (64,) -- (100, 64) -- (64, 64) -- (64,) -- (100, 64) -- (64, 64) -- (64,) -- (100, 64) -- (64, 64) -- (64,)
2 cls:Dropout               name:dropout_1             
3 cls:LSTM                  name:lstm_2                (64, 64) -- (64, 64) -- (64,) -- (64, 64) -- (64, 64) -- (64,) -- (64, 64) -- (64, 64) -- (64,) -- (64, 64) -- (64, 64) -- (64,)
4 cls:Dropout               name:dropout_2             
5 cls:LSTM                  name:lstm_3                (64, 64) -- (64, 64) -- (64,) -- (64, 64) -- (64, 64) -- (64,) -- (64, 64) -- (64, 64) -- (64,) -- (64, 64) -- (64, 64) -- (64,)
6 cls:Dropout               name:dropout_3             
7 

In [34]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop

# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [35]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [98]:
# -----------
if FN1 and os.path.exists('models/%s.h5'%FN1):
    model.load_weights('models/%s.h5'%FN1)
# -----------
FN1

'train'

In [99]:
model.get_weights()

[array([[-0.01773875,  0.06372642,  0.03280158, ..., -0.01024496,
         -0.0671009 , -0.05544017],
        [-0.08355927,  0.01300809, -0.03843831, ...,  0.05427392,
          0.05306476,  0.03605059],
        [-0.0115757 , -0.02824696,  0.06535474, ..., -0.02013939,
          0.08778789,  0.03364819],
        ...,
        [ 0.01810343,  0.02045769, -0.02760006, ..., -0.01984104,
          0.00498439,  0.02596883],
        [-0.05066778,  0.01373663,  0.01561437, ...,  0.08423416,
         -0.15663867,  0.01440313],
        [ 0.01368657, -0.08821091,  0.0784497 , ...,  0.01433412,
         -0.03293902, -0.02461588]], dtype=float32),
 array([[ 0.10238206, -0.11459416, -0.16845448, ..., -0.1220421 ,
          0.14774449, -0.06615916],
        [ 0.11321937, -0.08645081,  0.14741501, ...,  0.05086266,
         -0.00051563, -0.08621287],
        [-0.09718671, -0.18817723,  0.12291459, ..., -0.13177207,
         -0.11984888, -0.11752764],
        ...,
        [ 0.00524057,  0.0133075 ,  0.0

In [100]:
def lpadd(x, maxlen=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlen >= 0
    
    if maxlen == 0:
        return [eos]
    
    n = len(x)
    if n > maxlen:
        x = x[-maxlen:]
        n = maxlen
        
    return [empty]*(maxlen - n) + x + [eos]

In [101]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [102]:
np.all(data[:,maxlend] == eos)

True

In [103]:
print(data.shape,len(samples))

d = X_train[0]
d = [lpadd(d, maxlend)]
d = sequence.pad_sequences(d, maxlen=maxlen, value=empty, padding='post', truncating='post')
d.shape

(1, 144) 1


(1, 144)

In [104]:
probs = model.predict(data, verbose=1, batch_size=1)
probs

1/1 [==============================] - 0s


array([[[2.04325297e-05, 8.13607840e-05, 6.94496775e-05, ...,
         5.53332720e-05, 6.38350539e-05, 7.83460346e-05],
        [2.04325297e-05, 8.13607840e-05, 6.94496775e-05, ...,
         5.53332720e-05, 6.38350539e-05, 7.83460346e-05],
        [2.04325297e-05, 8.13607840e-05, 6.94496775e-05, ...,
         5.53332720e-05, 6.38350539e-05, 7.83460346e-05],
        ...,
        [2.04325297e-05, 8.13607840e-05, 6.94496775e-05, ...,
         5.53332720e-05, 6.38350539e-05, 7.83460346e-05],
        [2.04325297e-05, 8.13607840e-05, 6.94496775e-05, ...,
         5.53332720e-05, 6.38350539e-05, 7.83460346e-05],
        [2.04325297e-05, 8.13607840e-05, 6.94496775e-05, ...,
         5.53332720e-05, 6.38350539e-05, 7.83460346e-05]]])

In [105]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, 
               start=[empty]*maxlend + [eos],
               k=1, 
               maxsample=maxlen, 
               use_unk=True, 
               empty=empty,
               eos=eos, 
               temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [106]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = [len(i) for i in samples]
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in zip(probs, sample_lengths)])

In [107]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all occurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
    
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [108]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [109]:
import sys
import Levenshtein

def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):

    i = random.randint(0,len(X_test)-1)
#     print("y-test[i]: ", Y_test[i])
#     print("x-test[i]: ", X_test[i])
    
    print('HEAD:',' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print('DESC:',' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print('HEADS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)

    
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            
        print(score, ' '.join(words))
    

In [110]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: Middle Eastern women were once discouraged from sport new generation now chases Olympic glory
DESC: DUBAI United Arab Emirates coffeehouse^ with the world tallest building preening^ from two miles out the front window here sits another groundbreaker^ female Emirati Arab Muslim competitive weightlifter^ with vitality her tone hijab her head and herniated^ disk hollering^ from her lower back Not long ago Amna^ Haddad never would have guessed she would reach the spring 2016 speaking enthusiastic paragraphs about her unforeseen odyssey She never would have envisioned herself just off her national team attempt qualify for next month Olympics Rio Janeiro She certainly wouldn have imagined herself part concept gaining familiarity the Middle East the 2010s that the female athlete seeing unfold basically she said and athletes from seven nationalities plus those who raised them coach them know them agreed recent interviews They said there
HEADS:
163.66358463288677 buns 185 Sting appeal Act



Data generator generates batches of inputs and outputs/labels for training. The inputs are each made from two parts. The first maxlend words are the original description, followed by eos followed by the headline which we want to predict, except for the last word in the headline which is always eos and then empty padding until maxlen words.

For each, input, the output is the headline words (without the start eos but with the ending eos) padded with empty words up to maxlenh words. The output is also expanded to be y-hot encoding of each word.

To be more realistic, the second part of the input should be the result of generation and not the original headline. Instead we will flip just nflips words to be from the generator, but even this is too hard and instead implement flipping in a naive way (which consumes less time.) Using the full input (description + eos + headline) generate predictions for outputs. For nflips random words from the output, replace the original word with the word with highest probability from the prediction.


In [111]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
#     assert np.all(x[:,maxlend] == eos)
    
    probs = model.predict(x, verbose=0, batch_size=batch_size)

    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        if debug and b < debug:
            print(b)
            
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print('%s => %s'%(idx2word[x_out[b,input_idx]],idx2word[w]),)
            x_out[b,input_idx] = w
        if debug and b < debug:
            print()
    return x_out

In [112]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and headlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in zip(xds,xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')

    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [113]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
            
        new_seed = random.randint(0, 112233445)
        random.seed(c + new_seed)
        
        for b in range(batch_size):
            t = random.randint(0, len(Xd)-1)

            xd = Xd[t]
            xds.append(xd[:maxlend])
            
            xh = Xh[t]
            xhs.append(xh[:maxlenh])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [114]:
r = next(gen(X_train[:5], Y_train[:5], batch_size=batch_size))
r[0].shape, r[1].shape, len(r)


((128, 144), (128, 16, 40000), 2)

In [115]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)

In [116]:
test_gen(gen(X_train, Y_train, batch_size=batch_size))

L: Wall St. rises Dow racks seventh straight weekly gain ~ _ _ _ _ _ _ 
H: ~ Wall St. rises Dow racks seventh straight weekly gain _ _ _ _ _ _ 
D: The Dow Jones Industrial Average which briefly came within striking distance the historic 000 level earlier this week recorded its seventh straight weekly gain Following rally since the Nov. election the Dow about percent for the year and the 500 percent higher bets that the economy will benefit from Trump plans for deregulation and infrastructure spending Additional gains will depend how quickly and successfully Trump can deliver his promises and whether meets resistance from Congress reluctant widen the budget deficit many investors believe going get tough get over that 000 mark can but going take catalyst move investors said Jeff <0>^ regional investment director the Private Client Reserve Bank markets are shut for the Christmas holiday Monday After spending much the day with marginal loss the Dow ended 
L: Amber Heard files restraining o

In [117]:
test_gen(gen(X_train, Y_train, nflips=6, model=model, debug=False, batch_size=batch_size))

L: U.S. Women Hockey Team boycotting World Championships Protest Low Pay ~ _ _ _ _ _ 
H: ~ U.S. ~ Hockey ~ ~ World Championships Protest Low ~ _ _ _ _ _ 
D: Updated 10:15 Thursday The Women National Hockey Team the reigning world champions won defending their title this year They announced Wednesday that they will boycotting the championships later this month protest against USA Hockey citing stalled negotiations for fair wages and equitable support from the organization There may still women national team the ice though The boycotting players said statement Thursday that they have heard that USA Hockey attempting field alternative team play the World Championship <0>^ spokesman for USA Hockey says they still intend field team for the event but that they are currently speaking with the team basis will take look what the responses are and then will from there spokesman Dave Fischer told NPR The existing national team dominant player women ice hockey they <1>^ 
L: Cut Down <0>^ pesticide

In [118]:
valgen = gen(X_test, Y_test,nb_batches=3, batch_size=batch_size)

# TRAIN

In [119]:
history = {}

In [120]:
batch_size = 128
nb_val_samples_per_epoch = 1024

traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, nb_batches=nb_val_samples_per_epoch // batch_size, batch_size=batch_size)


In [121]:
r = next(traingen)
r[0].shape, r[1].shape, len(r)


((128, 144), (128, 16, 40000), 2)

In [122]:
history

{}

In [123]:
!ls -la 

total 800
drwxr-xr-x  23 MA573RWARR10R  staff     736 Nov  2 19:40 .
drwxr-xr-x@ 37 MA573RWARR10R  staff    1184 Oct 24 18:10 ..
-rw-r--r--@  1 MA573RWARR10R  staff    8196 Nov  2 18:17 .DS_Store
drwxr-xr-x  12 MA573RWARR10R  staff     384 Oct 22 22:19 .git
-rw-r--r--   1 MA573RWARR10R  staff     304 Oct 24 14:22 .gitignore
drwxr-xr-x   6 MA573RWARR10R  staff     192 Oct 29 13:30 .ipynb_checkpoints
-rw-r--r--   1 MA573RWARR10R  staff      13 Oct  9 22:58 README.md
-rw-r--r--   1 MA573RWARR10R  staff      78 Oct 24 14:42 __init__.py
drwxr-xr-x   8 MA573RWARR10R  staff     256 Nov  2 14:48 __pycache__
-rw-r--r--   1 MA573RWARR10R  staff      40 Oct 25 12:51 config.py
drwxr-xr-x  16 MA573RWARR10R  staff     512 Nov  2 18:35 data
-rw-r--r--@  1 MA573RWARR10R  staff    2108 Nov  2 13:52 data_loader.py
drwxr-xr-x   3 MA573RWARR10R  staff      96 Oct  9 12:12 glove
-rw-r--r--   1 MA573RWARR10R  staff     793 Oct 25 00:16 glove_loader.py
-rw-r--r--@  1 MA573RWARR10R  staff   88038 Oct 27 02:23

In [124]:
from keras.callbacks import ModelCheckpoint

filepath = 'models/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [125]:
print(lpadd(X_train[0], maxlend))
print(lpadd(Y_train[0], maxlenh))
print(lpadd(X_test[0], maxlend))
print(lpadd(Y_test[0], maxlenh))

[9, 17, 462, 4087, 5287, 36, 29697, 54, 5, 1485, 98875, 2, 2914, 740, 30206, 525, 2, 31158, 3, 2, 255, 274, 6380, 7, 17976, 11242, 46, 8, 69, 500, 991, 571, 7, 2, 20196, 43, 29, 8635, 20, 60, 2, 112, 840, 214, 5, 489, 17, 816, 349, 444, 571, 232, 2403, 2863, 1299, 456, 570, 29, 1297, 2, 16128, 3656, 537, 751, 51, 64, 248, 1098, 17, 64, 248, 8483, 34499, 1410, 999, 2468, 1357, 6894, 1410, 1970, 2031, 232, 16, 74, 119, 484, 24, 513, 629, 2, 50, 539, 339, 41, 34, 9, 83, 1408, 479, 484, 3266, 3, 5506, 1902, 484, 92, 33598, 83632, 32, 4, 16, 78, 111, 501, 148, 73, 8170, 49, 5311, 151, 113, 11, 1626, 18018, 15097, 5311, 1488, 15315, 1]
[0, 0, 0, 0, 6932, 6, 38199, 18095, 5228, 55226, 6, 10427, 1344, 11242, 48, 2914, 1]
[7, 3688, 4, 549, 1535, 175, 16681, 4, 611, 2065, 3678, 11056, 16794, 152, 549, 2347, 38480, 20, 1938, 28, 24, 9, 36, 128, 282, 2, 202, 386, 3076, 5, 16794, 777, 538, 44, 261, 20, 549, 2996, 2, 195, 683, 147, 63321, 9, 34, 1239, 488, 185, 741, 2065, 2, 333, 136, 5710, 4, 34, 1

In [126]:
for iteration in range(500):
    print('Iteration {}'.format(iteration))
    h = model.fit_generator(traingen, 
                            samples_per_epoch=nb_train_samples_per_epoch,
                            nb_epoch=1, 
                            validation_data=valgen, 
                            nb_val_samples=nb_val_samples_per_epoch,
                            callbacks=[checkpoint],)
    
    model.save_weights('data/%s.h5'%FN, overwrite=True)
    gensamples(batch_size=batch_size)

Iteration 0
Epoch 1/1
512/512 [==============================] - 148s - loss: 9.8833 - val_loss: 9.8908
HEAD: LISTEN Victim Doctors Describe^ War Zone Following Shootings^ Orlando
DESC: victim and his doctors described war zone following the deadliest mass public shooting modern United States history Dr. Chadwick^ Smith surgeon the Orlando Regional Medical Center Orlando Fla. said that little after Sunday patients began arriving into the emergency room was quickly filled capacity with people suffering with wounds the extremities^ the chest the pelvis and the abdomen Some had small wounds others had wounds Angel Colon was Pulse Orlando nightclub when the shooting happened all fresh said Tuesday that was hesitant share his story But said wants people know his story they can understand how heartless one person could Colon said that soon heard the gunshots and his friends ran But was shot three times ran hit the floor was trampled could hear the shots could hear
HEADS:
163.13306308835263 n

HEADS:
164.22495203688942 pseudoscience rattled tense Better misleading dynasty Eden Returns <1>^ Matter expendable investigate N.C. careless catered school
164.6312441596255 pseudoscience rattled tense Better misleading dynasty Eden Returns <1>^ Matter expendable investigate N.C. careless Acosta Principal
164.68989200814497 pseudoscience rattled tense Better misleading dynasty Eden Returns <1>^ Matter expendable investigate N.C. Westminster client Egypt
164.8838926875799 pseudoscience rattled tense Better misleading dynasty Eden Returns <1>^ Matter satellites Rays Sprouts falls signature Abbott
164.8968866917382 pseudoscience rattled tense Better misleading dynasty Eden Returns <1>^ Matter expendable investigate N.C. careless Acosta Sisters
165.1353595337997 pseudoscience rattled tense Better misleading dynasty Eden Returns <1>^ Victim Novak 1888 gamble eaglet names Tocqueville
165.33333422949966 pseudoscience rattled tense Better misleading dynasty Eden Returns <1>^ Matter expendable

HEADS:
159.0393459671726 turn extend People plan Bodies confirm that Maryland bellwether punch Allegations Cup pair St. begins Iran
159.37001668733978 turn extend People plan Bodies confirm that Maryland bellwether punch Allegations Cup pair St. begins merger
159.8722425771293 turn extend People plan Bodies confirm that Maryland bellwether punch Allegations Cup pair think northeast backed
160.17870110724832 turn extend People plan Bodies confirm that Maryland bellwether punch Allegations Cup strategy trained FBI minimums
160.214457574825 turn extend People plan Bodies confirm that Maryland bellwether punch Allegations Cup strategy trained FBI CMA
160.3135142238158 turn extend People plan Bodies confirm that Maryland bellwether punch Allegations Cup pair think assume Healthier
160.3645003979702 turn extend People plan Bodies confirm that Maryland bellwether punch Allegations Cup pair St. connect Passengers
161.45763663651155 turn extend People plan Bodies confirm that Maryland bellwethe

512/512 [==============================] - 144s - loss: 9.6764 - val_loss: 9.6954
HEAD: his inaugural address Trump leaves America better angels behind
DESC: FOR ANYONE^ who hoped that President Trump would use his inaugural address lower America political temperature expected that the rancor 2016 would give way elevated expressions more typical and appropriate Inauguration Day the new president remarks Friday can only described sharp disappointment Like his alarmist speech the Republican National Convention July this one painted false picture impoverished country that has been cheated and victimized Washington elites and grasping interests abroad Mr. Trump dystopia may exist places but not generally nation whose economy has rebounded from the recession and now outperforming other advanced industrial democracies Stoking^ discontent may serve Mr. Trump political interests but seems unlikely contribute the country stability unity purpose Nor will painting these purportedly unchecked ills

HEADS:
159.16928495451162 Jindal program. the epilogue abortion pregnancies virtual put Silk Carson Giants terrorism High Wade China Fake
159.51785553396863 fails University injuries praises demands <3>^ settlement everyone Tells Swarat dedicating debates momentum Deal until Guide
159.63918542682754 fails University injuries praises demands <3>^ settlement everyone Tells Swarat That depend program trendy 140 viruses
160.33780253252252 fails University injuries praises demands <3>^ settlement everyone Tells Swarat That Noonan really Louis knife pirate
160.44528425451924 fails University injuries praises demands <3>^ settlement everyone Tells Swarat That depend program trendy 140 Kalt
160.4621707659455 fails University injuries praises demands <3>^ settlement everyone Tells Swarat dedicating debates momentum Deal until alias
160.54171365783122 fails University injuries praises demands <3>^ settlement everyone Tells Swarat dedicating debates momentum Deal WASP known
160.90590702578226 fai

...

512/512 [==============================] - 143s - loss: 8.2467 - val_loss: 8.3486
HEAD: Japan Aso pushes back against U.S. pressure for bilateral trade deal
DESC: Japan has less room compromise with the United States under bilateral trade deal than under multilateral agreement like the Partnership TPP its deputy prime minister said taking swipe attempts directly pressure Tokyo into opening markets like agriculture Taro Aso who heads Japan for bilateral economic dialogue with the United States said that under TPP Japan was able accede^ more demands could make for the losses through agreements with other countries bilateral deal you can that You can get back what you lose from compromise with the United States Aso said seminar Columbia University Wednesday Aso comments underscore Japan hopes avoiding bilateral agreement FTA with the United States after President Donald Trump abandoned the TPP backed his predecessor Barack Obama and Japanese premier Shinzo Abe Tokyo fears agreement would


512/512 [==============================] - 141s - loss: 8.2515 - val_loss: 8.3676
HEAD: Trump wants kill federal arts funding What difference would that make
DESC: The wind Wilson Giant pinwheels^ and propellers start spinning atop tall and spindly^ kinetic sculptures called whirligigs^ which have been erected village green being developed into Whirligig^ Park The rotating wheels drive chains belts and shafts that turn set motion whimsical characters and shapes Little bicycle riders and unicyclists^ pedal and wave helicopters hover birds flap their wings fighter planes change course The fantastic contraptions^ have been fashioned from the discard pile American civilization freshly painted blue fan feet diameter spins majestically^ thanks the graceful repurposing the rear axle^ truck while another big pinwheel^ adorned with shiny metal milkshake^ cups Vollis^ Simpson the junkyard^ artist who built these figures worked from palette that also included scrap metal bicycle wheels attic vent

512/512 [==============================] - 141s - loss: 8.2545 - val_loss: 8.3685
HEAD: Every semi-competent^ male hero has ​more^ talented​^ female sidekick Why isn she the hero ​instead^
DESC: the season one finale The Magicians drama based the books about group students magical university there moment when the show protagonist begins doubt that the hero his own story Previously Quentin Coldwater^ Jason Ralph was entirely convinced his heroism had successfully traveled the magical land grown dreaming and that land god had laid its hands him and told him was the one who was meant save the world was the main character How could anything less than heroic course those who watched even few episodes the series read the books the series based will recall Quentin not the smartest the strongest the best magician his friends doesn have any special abilities like Penny who can teleport^ not his girlfriend Alice Olivia Taylor Dudley his best friend Julia Stella
HEADS:
65.22877078413545 voters be

512/512 [==============================] - 144s - loss: 8.2366 - val_loss: 8.3614
HEAD: Video shows Florida airport shooter open fire passengers scatter
DESC: The gunman who killed five people Florida airport walked calmly through the baggage claim area before wordlessly^ pulling handgun from his waistband and shooting victims who fled dived the floor panic video showed The seconds footage published Saturday website TMZ apparently from security camera shows the suspect carrying bag and clothing his left hand strolled with other passengers past luggage carousels^ Fort Lauderdale airport Without warning pulls out 9mm handgun with his right hand and fires repeatedly targets Travelers nearby can seen reacting horror and scrambling for cover the shots ring out TMZ did not say how obtained the video said had chosen show only the seconds leading the first shots fired and the panic that ensued.^ Esteban Santiago military veteran was arrested after running out ammunition and
HEADS:
113.52393567

512/512 [==============================] - 142s - loss: 8.2542 - val_loss: 8.3012
HEAD: Farewell Brangelina and the dream that love can last
DESC: Pack humanity Love just died Angelina Jolie has filed for divorce from Brad Pitt know pulling oversized cardigan letting its gigantic cuffs^ cover the tops hands and holding gigantic cup herbal tea Not drinking hate the stuff Perhaps later will get the tub listen some Sarah McLachlan^ and place glass red wine its edge remembering the good times eyes brimming seemed the naïve among the romantics^ that Brangelina were rare glimmer right world much wrong Who knows how why the whole enterprise worked just did Until sniff didn remember photo from the set and Mrs Smith Brad from the back all jaunty with one wee Maddox^ bottles his back pocket Just friends they were Brad was just lending hand Angelina was single mom having survived marriages one her
HEADS:
49.14094751742634 campaign new biggest With out order
105.25873493025904 campaign new biggest